In [1]:
from sentinelhub import SHConfig

config = SHConfig()
config.sh_client_id = 'sh-fe080851-d462-4304-9f4d-1b7fd7c7dc95'
config.sh_client_secret = 'XBnxUPfjs1duw9wmexDpoVmhTbY6piHw'
config.sh_base_url = 'https://sh.dataspace.copernicus.eu'
config.sh_token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'

In [2]:
import datetime
import os
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from rasterio.transform import from_bounds

from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)

# The following is not a package. Itis a file utils.py which should be in the same folder as this notebook.

In [3]:
from __future__ import annotations

from typing import Any

import matplotlib.pyplot as plt
import numpy as np


def plot_image(
    image: np.ndarray, factor: float = 1.0, clip_range: tuple[float, float] | None = None, **kwargs: Any
) -> None:
    """Utility function for plotting RGB images."""
    _, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 15))
    if clip_range is not None:
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
    else:
        ax.imshow(image * factor, **kwargs)
    ax.set_xticks([])
    ax.set_yticks([])

In [4]:
hyd_coords_wgs84 = (78.00405826,16.93264351,79.04971836,17.90150706)
resolution=(7000, 3500)
hyd_bbox = BBox(bbox=hyd_coords_wgs84, crs=CRS.WGS84)
hyd_size = bbox_to_dimensions(hyd_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {hyd_size} pixels")

Image shape at (7000, 3500) m resolution: (16, 30) pixels


In [5]:
import rasterio
from rasterio.transform import from_bounds
import os
import numpy as np

def generate_image(date_range, output_dir, filename, title, bbox=hyd_bbox, size=hyd_size, config=config):
    evalscript_mean_mosaic = """
    //VERSION=3
    function setup() {
        return {
            input: ["CO", "dataMask"],
            output: { 
                bands: 4 
            },
            mosaicking: "ORBIT" // To process data from different orbits
        };
    }

    const minVal = 0.02;
    const maxVal = 0.045;  // Adjust maxVal based on typical CO concentrations
    const diff = maxVal - minVal;

    const blueToRed = [
        [minVal, [0, 0, 1]],             // Blue
        [minVal + 0.14 * diff, [0, 0.5, 1]], // Blue-Cyan
        [minVal + 0.28 * diff, [0, 1, 1]],   // Cyan
        [minVal + 0.42 * diff, [0, 1, 0.5]], // Cyan-Green
        [minVal + 0.57 * diff, [0, 1, 0]],   // Green
        [minVal + 0.71 * diff, [1, 1, 0]],   // Yellow
        [minVal + 0.85 * diff, [1, 0.5, 0]], // Orange
        [maxVal, [1, 0, 0]]                  // Red
    ];

    const viz = new ColorRampVisualizer(blueToRed);

    function evaluatePixel(samples) {
        let totalCO = 0;
        let validSampleCount = 0;

        // Sum up the CO values and count valid samples
        for (let i = 0; i < samples.length; i++) {
            if (samples[i].dataMask === 1) {
                totalCO += samples[i].CO;
                validSampleCount++;
            }
        }

        // Calculate the mean CO value
        let meanCO = validSampleCount > 0 ? totalCO / validSampleCount : 0;

        // If no valid samples, return white color
        if (validSampleCount === 0) {
            return [1, 1, 1, 1]; // White with full opacity
        }

        // Apply the colormap visualization for values within the range
        let rgba = viz.process(meanCO);
        rgba.push(1); // Set alpha channel to 1 (fully opaque)

        return rgba;
    }
    """

    request = SentinelHubRequest(
        evalscript=evalscript_mean_mosaic,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL5P.define_from(
                    "s5pl2", service_url=config.sh_base_url
                ),
                time_interval=date_range,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
        bbox=bbox,
        size=size,
        config=config,
    )

    image = request.get_data()[0]

    # Get the bounding box coordinates
    min_x, min_y, max_x, max_y = bbox.min_x, bbox.min_y, bbox.max_x, bbox.max_y

    # Define the affine transform
    transform = from_bounds(min_x, min_y, max_x, max_y, image.shape[1], image.shape[0])

    # Define the coordinate reference system (CRS)
    crs = {"init": "EPSG:4326"}  # Assuming WGS84 coordinates

    # Save the image as a GeoTIFF with georeferencing information
    output_file = os.path.join(output_dir, filename + ".tif")
    with rasterio.open(
        output_file,
        "w",
        driver="GTiff",
        height=image.shape[0],
        width=image.shape[1],
        count=4,  # RGBA
        dtype=image.dtype,
        crs=crs,
        transform=transform,
        compress='none'
    ) as dst:
        # Write each band separately
        for i in range(4):  # 4 bands (RGBA)
            dst.write(image[:, :, i], i + 1)

    print(f"Image saved as GeoTIFF with georeferencing at {output_file}")


In [6]:
#generate_image(("2021-1-16", "2021-1-25"),r"C:\Users\rishi\ml_projects\climate\to_delete","waste","Mean CO")

In [7]:
months = {
    1: ("January", 31),
    2: ("February", 28),
    3: ("March", 31),
    4: ("April", 30),
    5: ("May", 31),
    6: ("June", 30),
    7: ("July", 31),
    8: ("August", 31),
    9: ("September", 30),
    10: ("October", 31),
    11: ("November", 30),
    12: ("December", 31)
}

for year in range(2018,2024):
    for i in range(1,13):
        month_name, end_day = months[i]
        start=f"{year}-{i}-1"
        end = f"{year}-{i}-{end_day}"
        path = f"{year}_{month_name}"
        title = f"{year}_{month_name}"
        
        generate_image(
            date_range=(start, end),
            output_dir=r"C:\Users\rishi\ml_projects\climate\Intermediate outputs\CO_images_all",
            filename=path,
            title=title
        )



Image saved as GeoTIFF with georeferencing at C:\Users\rishi\ml_projects\climate\Intermediate outputs\CO_images_all\2018_January.tif
Image saved as GeoTIFF with georeferencing at C:\Users\rishi\ml_projects\climate\Intermediate outputs\CO_images_all\2018_February.tif
Image saved as GeoTIFF with georeferencing at C:\Users\rishi\ml_projects\climate\Intermediate outputs\CO_images_all\2018_March.tif
Image saved as GeoTIFF with georeferencing at C:\Users\rishi\ml_projects\climate\Intermediate outputs\CO_images_all\2018_April.tif
Image saved as GeoTIFF with georeferencing at C:\Users\rishi\ml_projects\climate\Intermediate outputs\CO_images_all\2018_May.tif
Image saved as GeoTIFF with georeferencing at C:\Users\rishi\ml_projects\climate\Intermediate outputs\CO_images_all\2018_June.tif
Image saved as GeoTIFF with georeferencing at C:\Users\rishi\ml_projects\climate\Intermediate outputs\CO_images_all\2018_July.tif
Image saved as GeoTIFF with georeferencing at C:\Users\rishi\ml_projects\climate\I